# Plotting Overview

## Getting started

`scipp` offers a number of different ways to plot data from a `DataArray` or a `Dataset`.
It uses the `matplotlib` graphing library to do so, as well as the `pythreejs` project for 3D visualizations.

Plotting functionality is available in two different ways:
- using the `plot()` free function
- using the `.plot()` method on a Scipp object (variable, data array or dataset)

The difference between the two possible plot functions is that the free function can accept more input types than just the Scipp objects.
It can also plot raw numpy arrays, as well as python dicts of Scipp variables or data arrays.
For Scipp objects, the produced plot will be the same with either approach: 
Internally, the `.plot()` method just forwards the Scipp object to the free function `plot()`.

In [ ]:
import numpy as np
import scipp as sc

In [ ]:
N = 20
M = 5
d = sc.Dataset(
    data={
        'data':sc.array(dims=['y'], values=100*np.random.rand(N)+50, unit='counts'),
        'data_with_errors':sc.array(dims=['y'],
                                    values=50*np.random.rand(N) + 20.,
                                    variances=50*np.random.rand(N),
                                    unit='counts'),
        'data_2d':sc.array(dims=['x', 'y'], values=10.0*np.random.rand(M, N), unit=sc.units.K)},
    coords={
        'x':sc.array(dims=['x'], values=np.arange(M), unit=sc.units.m),
        'y':sc.array(dims=['y'], values=np.arange(N), unit=sc.units.us)} )

The information in a data array or dataset is typically enough to create meaningful plots:

In [ ]:
sc.plot(d)

Notice in above that the dataset contains three data items.
The two 1-D items with dimension y are combined into a single 1-D plot.
The third item is 2-D and plotted separately.

All plots will use dimension coordinates values for each axis by default if they are available.
For the plot above, x and y dimensions have associated coordinates.
Notice that the units on the axes are also taken from the coordinates.

## Plotting slices or items of a dataset

The usual indexing and slicing can be used to create plots of slices of data, or plots of individual items from a dataset.

### Plot a single entry of a dataset

In [ ]:
sc.plot(d['data'])

or alternatively

In [ ]:
d['data'].plot()

### Plot a slice range

In [ ]:
sc.plot(d['y', 4:7])

### Plot a 1-D slice of 2-D data

When slicing without a range, the dimensionality reduces.
This can be used to, e.g., plot a 1-D slice through 2-D data:

In [ ]:
sc.plot(d['x', 4])

## Logarithmic scale

1-D data can be plotted on a logarithmic scale on one or both axes:

In [ ]:
sc.plot(d, scale={'y': 'log'})

## Axis labels and axis order

By default scipp uses coordinate values to label the axes.
If a data array or dataset contains auxiliary coordinates, these can be used instead.
This is configured using the `labels` keyword argument of `plot`:

In [ ]:
d.coords['xlabels'] = sc.Variable(dims=['x'], values=np.arange(M) + 15.)
sc.plot(d['data_2d'], labels={'x': 'xlabels'})

<div class="alert alert-info">

*Note*

The `labels` dict uses *dimension labels* as keys, i.e., the `'x'` above refers to the `x` dimension and *not* the x-axis of the matplotlib figure.

</div>

This also works for attributes:

In [ ]:
d['data_2d'].attrs['T'] = sc.Variable(dims=['x'], values=np.linspace(3, 50, M), unit=sc.units.K )
sc.plot(d['data_2d'], labels={'x': 'T'})

To control which dimensions are shown along which axes of the matplotlib figure use the transpose button in the toolbar, or transpose data before calling `plot`, e.g.

In [ ]:
d['data_2d'].transpose().plot()

<div class="alert alert-info">

*Note*

Plot settings are cached in a scipp `config.yaml` file.
See <a href="../python-reference/runtime-configuration.ipynb">runtime-configuration</a> for more information

</div>